# Performance and Valuation Insights: A Comprehensive Analysis of German Bundesliga Clubs with FC Bayern Munich as a Case Study (Season 2023/2024)

This portfolio project provides an in-depth analysis of FC Bayern Munich's performance and market dynamics during the Bundesliga 2023-2024 season. Leveraging Python for data manipulation and analysis, this project highlights key trends and derives actionable insights.

### Author: Moritz Philipp Haaf, BSc MA

### Contact Information:
- **Email:** moritz_haaf@outlook.com
- **GitHub:** [itzmore-mph/itzmore-mph-portfolio](https://github.com/itzmore-mph/itzmore-mph-portfolio)

## 1. Initial Setup
Import libraries for SQL, data manipulation, visualization, and regex.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import duckdb
import warnings
warnings.filterwarnings('ignore')
print("Working directory:", os.getcwd())

## 2. Load Data with DuckDB
Register CSV files as DuckDB views for SQL querying.

In [ ]:
data_path = './Raw-Data_CSV_Football-Analysis_German-Bundesliga/'

con = duckdb.connect()
tables = ['players','player_valuations','game_events','appearances','clubs','club_games','competitions','games']
for table in tables:
    con.execute(f"CREATE VIEW {table} AS SELECT * FROM read_csv_auto('{data_path}{table}.csv')")

print('DuckDB views created:', tables)

## 3. SQL Data Exploration
Retrieve average market values and total goals/xG per player.

In [ ]:
# 3.1 Average Market Value per Player
avg_val_query = '''
SELECT p.player_name AS player, AVG(pv.market_value) AS avg_value
FROM players p
JOIN player_valuations pv USING(player_id)
GROUP BY player
ORDER BY avg_value DESC
LIMIT 10;
'''
avg_val = con.execute(avg_val_query).df()
avg_val

**Interpretation:** These top 10 players by average market value highlight the most valuable talents over the season.

In [ ]:
# 3.2 Total Goals and xG per Player
player_stats_query = '''
SELECT p.player_name AS player,
       SUM(ge.goals) AS total_goals,
       SUM(ge.xG)    AS total_xG
FROM players p
JOIN game_events ge USING(player_id)
GROUP BY player
ORDER BY total_goals DESC
LIMIT 10;
'''
player_stats = con.execute(player_stats_query).df()
player_stats

**Interpretation:** Shows which players delivered the highest goal output and how it relates to their expected goals.

## 4. Load & Preprocess Data with Pandas
Standardize column names, parse dates, validate keys, and create Bayern subset.

In [ ]:
# 4.1 Load CSVs with Pandas
appearances       = pd.read_csv(data_path + 'appearances.csv')
club_games        = pd.read_csv(data_path + 'club_games.csv')
clubs             = pd.read_csv(data_path + 'clubs.csv')
competitions      = pd.read_csv(data_path + 'competitions.csv')
game_events       = pd.read_csv(data_path + 'game_events.csv')
games             = pd.read_csv(data_path + 'games.csv')
player_valuations = pd.read_csv(data_path + 'player_valuations.csv')
players           = pd.read_csv(data_path + 'players.csv')


In [ ]:
# 4.2 Helper: snake_case conversion
def to_snake(col: str) -> str:
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', col)
    s2 = re.sub('([a-z0-9])([A-Z])',      r'\1_\2', s1)
    s3 = s2.replace(' ', '_').lower()
    return re.sub(r'__+','_', s3)

for df in [players, player_valuations, game_events, appearances, clubs, club_games, competitions, games]:
    df.columns = [to_snake(c) for c in df.columns]

**Interpretation:** Consistent snake_case naming ensures code clarity and easier maintenance.

In [ ]:
# 4.3 Parse datetime columns
player_valuations['date'] = pd.to_datetime(player_valuations['date'], errors='coerce')
games['date']             = pd.to_datetime(games['date'], errors='coerce')

**Note:** Any invalid date strings become `NaT` for easy detection.

In [ ]:
# 4.4 Cleaning & Validation
def prepare_data(df):
    df = df.copy()
    df.fillna(0, inplace=True)
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
    return df

# Apply to main tables
players           = prepare_data(players)
player_valuations = prepare_data(player_valuations)
game_events       = prepare_data(game_events)

# 4.5 Key validation: find orphan player_ids
ev_ids    = set(game_events['player_id'])
pl_ids    = set(players['player_id'])
orphans   = ev_ids - pl_ids
if orphans:
    print(f"{len(orphans)} orphan player_id(s) in game_events:", list(orphans)[:10])


**Interpretation:** Early detection of mismatched keys prevents silent data errors.

In [ ]:
# 4.6 Subset for FC Bayern Munich (club_id = 27)
bayern_stats = game_events[game_events['club_id'] == 27]
bayern_stats = prepare_data(bayern_stats)
print(f'Bayern events: {len(bayern_stats)} rows')

**Interpretation:** Working on the Bayern subset allows focused analysis on the case study.

## 5. Exploratory Data Analysis (EDA)
Visualize key relationships in the Bayern subset.

### 5.1 Correlation: xG vs Goals

In [ ]:
# Drop any missing for clarity
clean_evt = bayern_stats.dropna(subset=['xg','goals'])
plt.figure(figsize=(10,6))
sns.scatterplot(data=clean_evt, x='xg', y='goals')
plt.title('FC Bayern: Expected Goals (xG) vs Actual Goals')
plt.xlabel('xG')
plt.ylabel('Goals')
plt.show()

**Interpretation:** Points above the diagonal indicate matches where players outperformed their expected goals.

## 6. Performance Analysis

### 6.1 Match Result Distribution

In [ ]:
def performance_analysis(df):
    counts = df['result'].value_counts().reset_index()
    counts.columns = ['result','count']
    plt.figure(figsize=(8,5))
    sns.barplot(x='result', y='count', data=counts)
    plt.title('FC Bayern Match Results (W/D/L)')
    plt.xlabel('Result')
    plt.ylabel('Matches')
    plt.show()

performance_analysis(bayern_stats)

**Interpretation:** The win/draw/loss distribution highlights overall season performance strengths.

## 7. Market Value Analysis

### 7.1 Market Value Trend

In [ ]:
fig = px.line(player_valuations, x='date', y='market_value', title='Market Value Trend of All Players', 
                 labels={'market_value':'Market Value','date':'Date'})
fig.update_layout(xaxis_title='Date', yaxis_title='Market Value')
fig.show()

**Interpretation:** Visualizes the investment dynamics and how player valuations evolve over time.

## 8. Predictive Modeling

### 8.1 Linear Regression to Predict Market Value

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

X = player_valuations[['goals','assists','minutes_played']]
y = player_valuations['market_value']

model = Ridge(alpha=1.0)
scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f'5-Fold CV R² scores: {scores.round(2)}')
print(f'Average R²: {scores.mean():.2f}')

**Interpretation:** Regularized regression and cross-validation provide more robust estimates of predictive power.

## 9. Expected Goals & Cumulative Analysis

### 9.1 Cumulative xG vs Goals Over Season

In [ ]:
# Merge xG and goals in player_valuations for cumulative over time
player_valuations['xg']   = np.nan  # ensure column exists
player_valuations['goals'] = np.nan
# fill from global game_events
agg = game_events.groupby('date').agg({'xg':'sum','goals':'sum'}).reset_index()
pv_time = player_valuations.merge(agg, on='date', how='left').fillna(0)
pv_time = pv_time.sort_values('date')

pv_time['cum_xg']    = pv_time['xg'].cumsum()
pv_time['cum_goals'] = pv_time['goals'].cumsum()

plt.figure(figsize=(12,7))
plt.plot(pv_time['date'], pv_time['cum_xg'],    label='Cumulative xG')
plt.plot(pv_time['date'], pv_time['cum_goals'], label='Cumulative Goals')
plt.legend()
plt.title('Cumulative xG vs Actual Goals Over Season')
plt.xlabel('Date')
plt.ylabel('Total')
plt.show()

**Interpretation:** Tracking cumulative metrics reveals overall scoring efficiency trends.

## Conclusion
**Key Takeaways:**
- Bayern’s players often outperformed xG, indicating strong finishing.
- Season win percentage underscores dominance in match results.
- Market-value trends align with performance peaks.
- A regularized model achieves stable predictive performance (avg R²).  

**Future Directions:**
- Incorporate additional features (e.g., assists types, distance covered).
- Build an interactive dashboard (Streamlit or Voilà).
- Extend analysis to other clubs for comparative insights.